<a href="https://colab.research.google.com/github/vicente-gonzalez-ruiz/denoising/blob/main/src/denoising/SRA_volume_denoising_opticalflow3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SRA_empiar10311

In [ ]:
#%pip install ipynbname

In [ ]:
#%pip install gdown

In [ ]:
#%pip install matplotlib

In [ ]:
# %pip install --force-reinstall "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
#%pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
#%pip install cupy-cuda12x # Check the CUDA installed with nvcc --version (that should match with the version provided by nvidia-smi)
#%pip install numba

In [ ]:
#%pip install --force-reinstall "motion_estimation @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"
#%pip uninstall -y motion_estimation
#%pip install "motion_estimation @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"

In [ ]:
#%pip install --force-reinstall "optical_flow_3D @ git+https://github.com/vicente-gonzalez-ruiz/optical_flow_3D"
#%pip uninstall -y optical_flow_3D
#%pip install "optical_flow_3D @ git+https://github.com/vicente-gonzalez-ruiz/optical_flow_3D"

In [ ]:
import mrcfile
import numpy as np
import os
import mrcfile
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from collections import namedtuple
import logging
import motion_estimation._3D.farneback_optical_flow_3D as farneback3D
import denoising.volume.SRA_optical_flow_3D as SRA
import motion_estimation._3D.project_optical_flow_3D as project
from PIL import Image
import ipynbname

In [ ]:
vol_filename = "empiar10311_stack_crop.mrc"
Args = namedtuple("args", ["input", "output"])
fn, fe = vol_filename.split(".")
args = Args(vol_filename , fn + "_denoised." + fe)
print(args)

In [ ]:
%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1lSFb4VZ5iqR3kLdPv1dDbdOe9dKPZ--_" #https://drive.google.com/file/d/1lSFb4VZ5iqR3kLdPv1dDbdOe9dKPZ--_/view?usp=drive_link
    #FILEID="1YDyHWyB1d6yq2LYif2zQcQgr7vhlspYo" # https://drive.google.com/file/d/1YDyHWyB1d6yq2LYif2zQcQgr7vhlspYo/view?usp=sharing
    #FILEID="1iui018SGKa5nb0ybeoUAB7uCUhl9EJ5o" #https://drive.google.com/file/d/1iui018SGKa5nb0ybeoUAB7uCUhl9EJ5o/view?usp=sharing
    #wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME #2> /dev/null
    #wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=FILEID" -O FILENAME && rm -rf /tmp/cookies.txt
    ~/envs/OF3D_CUDA/bin/gdown https://drive.google.com/uc?id=$FILEID # pip install gdown
fi
ls -l $OUTPUT_FILENAME
set +x

In [ ]:
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.setLevel(logging.WARNING)

In [ ]:
!ln -s ~/repos/denoising/src/denoising .

In [ ]:
OF_estimator = farneback3D.OF_Estimation(logger)

In [ ]:
projector = project.Project(logger)

In [ ]:
#%pip install mrcfile
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data
noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)
print(noisy.shape)

In [ ]:
def get_notebook_name():
    ip = get_ipython()
    this_notebook_path = ip.user_ns['__session__']
    this_notebook_name = os.path.basename(this_notebook_path)
    return this_notebook_name, this_notebook_path

In [ ]:
get_notebook_name()

In [ ]:
def show_image(denoised, title):
    slice_idx = denoised.shape[0]//2
    figure(figsize=(32, 32))
    plt.subplot(1, 2, 1)
    plt.title("denoised")
    plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
    plt.subplot(1, 2, 2)
    plt.title("difference")
    plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")
    plt.show()

In [ ]:
class Output_Controller:

    def __init__(self):
        self.image_counter = 0
    
    def show_image(self, denoised, title):
        slice_idx = denoised.shape[0]//2
        image_float = denoised[slice_idx]
        image_8b = (255*(image_float - np.min(image_float))/(np.max(image_float) - np.min(image_float))).astype(np.uint8)
        image = Image.fromarray(image_8b)
        notebook_name = get_notebook_name()[0]
        output_filename = f"{notebook_name}_{self.image_counter}.png"
        image.save(output_filename)
        self.image_counter += 1

OC = Output_Controller()

In [ ]:
denoiser = SRA.Shuffle_Register_and_Average(
    OF_estimator,
    projector,
    logger,
    quality_estimator=None,
    show_image=OC.show_image)

In [ ]:
denoised = denoiser.filter(
    noisy_volume=noisy,
    N_iters=25,
    std_dev=2.0,
    pyramid_levels=3,
    gauss_size=11,
    iterations=5,
    sigma_k=0.45,
    #filter_type="box", # box | gaussian
    filter_type="gaussian", # box | gaussian
    filter_size=9, # std_dev=2.0
    presmoothing=None
)

In [ ]:
with mrcfile.new(f"{fn}_SRA_denoised.mrc", overwrite=True) as mrc:
    mrc.set_data(denoised.astype(np.float32))
    mrc.data

In [ ]:
denoised = mrcfile.open(f"{fn}_SRA_denoised.mrc").data

In [ ]:
denoised.shape

In [ ]:
figure(figsize=(16, 16))
slice_idx = denoised.shape[0]//2
plt.imshow(denoised[slice_idx, 200:600, 200:600], cmap="gray")

In [ ]:
figure(figsize=(16, 16))
slice_idx = noisy.shape[0]//2
plt.imshow(noisy[slice_idx, 200:600, 200:600], cmap="gray")

In [ ]:
import information_theory
slice_index = noisy.shape[0]//2
fig, axs = plt.subplots(1, 3, figsize=(15, 45))
axs[0].imshow(noisy[16], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[slice_index], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[slice_index], denoised[slice_index])})")
axs[2].imshow(noisy[slice_index] - denoised[slice_index], cmap="gray")
axs[2].set_title(f"Noisy - Denoised")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[1]//2
axs[0].imshow(noisy[:, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, slice_idx], denoised[:, slice_idx])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[2]//2
axs[0].imshow(noisy[:, :, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, :, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, :, slice_idx], denoised[:, :, slice_idx])})")
fig.tight_layout()
plt.show()

In [ ]:
denoiser = SRA.Shuffle_Register_and_Average(OF_estimator, projector, logger, quality_estimator=None, show_image=show_image)

In [ ]:
denoised = denoiser.filter(
    noisy_volume=noisy,
    N_iters=25,
    std_dev=2.0,
    pyramid_levels=3,
    gauss_size=11,
    iterations=5,
    sigma_k=1.5,
    #filter_type="box", # box | gaussian
    filter_type="gaussian", # box | gaussian
    filter_size=9, # std_dev=2.0
    presmoothing=None
)

In [ ]:
figure(figsize=(16, 16))
slice_idx = denoised.shape[0]//2
plt.imshow(denoised[slice_idx, 200:600, 200:600], cmap="gray")

In [ ]:
denoiser = SRA.Shuffle_Register_and_Average(OF_estimator, projector, logger, quality_estimator=None, show_image=show_image)

In [ ]:
denoised = denoiser.filter(
    noisy_volume=noisy,
    N_iters=25,
    std_dev=2.0,
    pyramid_levels=3,
    gauss_size=3,
    iterations=5,
    sigma_k=0.5,
    #filter_type="box", # box | gaussian
    filter_type="gaussian", # box | gaussian
    filter_size=9, # std_dev=2.0
    presmoothing=None
)

In [ ]:
figure(figsize=(16, 16))
slice_idx = denoised.shape[0]//2
plt.imshow(denoised[slice_idx, 200:600, 200:600], cmap="gray")

In [ ]:
input()

In [ ]:
#import opticalflow3D
#import warnings
#from numba.core.errors import NumbaPerformanceWarning
#warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)
#else:
#    import motion_estimation._3D.farneback as farneback3D

In [ ]:
#%pip install "optical_flow_3D @ git+https://github.com/vicente-gonzalez-ruiz/optical_flow_3D"

In [ ]:
#%pip install --force-reinstall "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
#%pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
#%pip uninstall -y information_theory
import information_theory  # pip install --force-reinstall "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
if use_opticalflow3D:
    block_size = (noisy.shape[0], noisy.shape[1], noisy.shape[2])

In [ ]:
if use_opticalflow3D:
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=33, presmoothing=None, filter_type="gaussian", filter_size=3)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=13, presmoothing=None, filter_type="gaussian", filter_size=7)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=5, presmoothing=3, filter_type="gaussian", filter_size=5)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=3, presmoothing=5, filter_type="box", filter_size=3)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=3, presmoothing=5, filter_type="box", filter_size=3)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=3, presmoothing=5, filter_type="gaussian", filter_size=11)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=5, presmoothing=4, filter_type="box", filter_size=5)
    estimator = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=9, sigma_k=0.15, filter_type="gaussian" ,filter_size=7, presmoothing=None, device_id=0)
else:
    estimator = farneback3D(

In [ ]:
RS_sigma = 0.75
if use_opticalflow3D:
    denoised = RSIVD.filter(farneback, block_size, noisy, RS_sigma=RS_sigma, N_iters=20)
else:


In [ ]:
np.min(denoised)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, 100], denoised[:, 100])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, :, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, :, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, :, 100], denoised[:, :, 100])})")
fig.tight_layout()
plt.show()

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15][::-1, :] - denoised_vol[15][::-1, :], cmap="gray")

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15][::-1, :] - denoised_vol[15][::-1, :], cmap="gray")

In [ ]:
skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)

In [ ]:
img = skimage.io.imread("http://www.hpca.ual.es/~vruiz/images/barb.png")#[:256, :256]

In [ ]:
noisy_vol = np.stack([img]*32)

In [ ]:
noisy_vol.shape

In [ ]:
mean = 0
var = 1000
sigma = var**0.5
for i in range(noisy_vol.shape[0]):
    noise = np.random.normal(mean,sigma,img.shape).reshape(img.shape)
    noisy_vol[i] = np.clip(a=img.astype(np.float32) + noise, a_min=0, a_max=255).astype(np.uint8)

In [ ]:
farneback = opticalflow3D.Farneback3D(iters=5,
                                      num_levels=3,
                                      scale=0.5,
                                      spatial_size=5,
                                      presmoothing=2,
                                      filter_type="box",
                                      filter_size=5,
                                     )

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=30)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=100)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
RS_sigma = 2.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=30)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
RS_sigma = 3.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=30)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
#skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
if IN_COLAB:
    print("Running in Colab")
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2

In [ ]:
if IN_COLAB:
    !pip install cupy-cuda12x
    !pip install opticalflow3D

In [ ]:
#use_opticalflow3D = False

In [ ]:
#%pip install opticalflow3d

In [ ]:
#vol_name = "small_vol"

In [ ]:
if IN_COLAB:
    print("Running in Colab")
    !pip install cupy-cuda12x
    !pip install opticalflow3D
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2
    from google.colab import drive
    drive.mount('/content/drive')
    !cp drive/Shareddrives/TomogramDenoising/tomograms/{vol_name}.tif .
else:
    print("Running in locahost")
    !cp ~/Downloads/{vol_name}.tif .